# **Teacher Learner Based Optimization (TLBO)**

Name   : ABHISHEK SINGH

In [73]:
import numpy as np

# TLBO algorithm
def TLBO(pop, f_obj, f_cons):
    
    best = pop[np.argmin([f_obj(individual) for individual in pop])]
    
    for i in range(max_iter):
        # Teacher phase
        teacher = best.copy()
        for j in range(n_dim):
            other_indices = [k for k in range(n_pop) if k != j]
            mean_others = np.mean(pop[other_indices], axis=0)
            if abs(mean_others[j] - teacher[j]) < np.finfo(float).eps:
                delta_j = np.random.uniform(0, 1)
            else:
                delta_j = np.random.uniform(0, 1) * abs(mean_others[j] - teacher[j])
            if mean_others[j] < teacher[j]:
                teacher[j] = teacher[j] - delta_j
            else:
                teacher[j] = teacher[j] + delta_j
        
        # Learner phase
        for j in range(n_pop):
            learner = pop[j].copy()
            a = np.random.randint(n_pop)
            while a == j:
                a = np.random.randint(n_pop)
            b = np.random.randint(n_pop)
            while b == j or b == a:
                b = np.random.randint(n_pop)
            for k in range(n_dim):
                if np.random.uniform(0, 1) < 0.5:
                    learner[k] = learner[k] + np.random.uniform(0, 1) * (teacher[k] - np.random.uniform(0, 1) * abs(2 * pop[a][k] - pop[b][k]))
                else:
                    learner[k] = learner[k] - np.random.uniform(0, 1) * (teacher[k] - np.random.uniform(0, 1) * abs(2 * pop[a][k] - pop[b][k]))
            learner = np.clip(learner, -1, 1)
            
            # Apply constraints
            c = f_cons(learner)
            if np.any(c > 0):
                for k in range(len(c)):
                    if c[k] > 0:
                        learner[k] = learner[k] - c[k]
                        learner[k] = np.clip(learner[k], -1, 1)
            
            # Update the population
            if f_obj(learner) < f_obj(pop[j]): # For maximization reverse the sign
                pop[j] = learner.copy()
                if f_obj(learner) < f_obj(best): # For maximization reverse the sign
                    best = learner.copy()
    
    return best

In [69]:
# Benchmark 1 - quadratic minimization with 13 variables and 9 linear inequality constraints
def objective_1(x):
    return 5 * sum(x[:4]) - 5 * sum(x[:4]**2) - sum(x[5:13])

# Inequality constraint functions
def constraint_1(x):
    constraints = np.zeros(len(x))
    constraints[0] = 2 * x[0] + 2 * x[1] + x[9] + x[10] - 10
    constraints[1] = 2 * x[0] + 2 * x[2] + x[9] + x[11] - 10
    constraints[2] = 2 * x[1] + 2 * x[2] + x[10] + x[11] - 10
    constraints[3] = (- 8 * x[0]) + x[9]
    constraints[4] = (- 8 * x[1]) + x[10]
    constraints[5] = (- 8 * x[2]) + x[11]
    constraints[6] = (- 2 * x[3]) - x[4] + x[9]
    constraints[7] = (- 2 * x[5]) - x[6] + x[11]
    constraints[8] = (- 2 * x[7]) - x[8] + x[11]
    return constraints

# Initialize
max_iter = 500 # No. of generations
n_pop = 50 # Population size
n_dim = 13 # No. of design variables
pop = np.random.uniform(0, 1, size=(n_pop, n_dim)) # Range

t1 = TLBO(pop, f_obj = objective_1, f_cons = constraint_1)

print("No. of generation: ", max_iter)
print("Population Size: ", n_pop)
print("No. of design variables: ", n_dim)
print("\nBest Student found:")
print(t1)
fitness_value = objective_1(t1)
print("\nFitness of best Student:", fitness_value)

No. of generation:  500
Population Size:  50
No. of design variables:  13

Best Student found:
[-1.         -1.         -1.         -1.         -1.         -1.
 -1.          0.64633084  1.          1.          0.94162636  0.652017
  0.94814267]

Fitness of best Student: -43.18811687108179


In [68]:
# Benchmark 2 - nonlinear maximization with 10 variables and 1 non linear equality constraint
def objective_2(x):
    for i in range(len(x)):
      return np.prod(x * np.sqrt(i)**i)

# Inequality constraint functions
def constraint_2(x):
    constraints = np.zeros(len(x))
    constraints[0] = sum(x**2) - 1
    return constraints

# Initialize
max_iter = 2000 # No. of generations
n_pop = 50 # Population size
n_dim = 10 # No. of design variables
pop = np.random.uniform(0, 10, size=(n_pop, n_dim)) # Range

t2 = TLBO(pop, f_obj = objective_2, f_cons = constraint_2)

print("No. of generation: ", max_iter)
print("Population Size: ", n_pop)
print("No. of design variables: ", n_dim)
print("\nBest Student found:")
print(t2)
fitness_value = objective_2(t2)
print("\nFitness of best Student: ", fitness_value)

No. of generation:  2000
Population Size:  50
No. of design variables:  10

Best Student found:
[-1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]

Fitness of best Student:  -1.0


In [70]:
# Benchmark 3 - nonlinear minimization with 7 variables and 4 non linear inequality constraints
def objective_3(x):
    return ((x[0] - 10)**2) + (5 * (x[1] - 12)**2) + x[2]**4 + (3 * (x[3] - 11)**2) + (10 * x[4]**6) + 7 * x[5]**2 + x[6]**4 - (4 * (x[5] * x[6])) - 10 * x[5] - 8 * x[6]

# Inequality constraint functions
def constraint_3(x):
    constraints = np.zeros(len(x))
    constraints[0] = -127 + 2 * x[0]**2 + 3 * x[1]**4 + x[2] + 4 * x[3]**4 + 5 * x[4]
    constraints[1] = -282 + 7 * x[0] + 3 * x[1] + 10 * x[2]**2 + x[3] - x[4]
    constraints[2] = -196 + 23 * x[0] * (x[1]**2) + 6 * x[5]**2 - 8 * x[6]
    constraints[3] = 4 * x[0]**2 + x[1]**2 - 3 * x[0] * x[1] + 2 * x[2]**2 + 5 * x[5] - 11 * x[6]
    return constraints

# Initialize
max_iter = 2000 # No. of generations
n_pop = 50 # Population size
n_dim = 7 # No. of design variables
pop = np.random.uniform(-10, 10, size=(n_pop, n_dim)) # Range

t3 = TLBO(pop, f_obj = objective_3, f_cons = constraint_3)

print("No. of generation: ", max_iter)
print("Population Size: ", n_pop)
print("No. of design variables: ", n_dim)
print("\nBest Student found:")
print(t3)
fitness_value = objective_3(t3)
print("\nFitness of best Student:", fitness_value)

No. of generation:  2000
Population Size:  50
No. of design variables:  7

Best Student found:
[1.00000000e+00 1.00000000e+00 3.92786298e-04 1.00000000e+00
 2.95454033e-03 1.00000000e+00 1.00000000e+00]

Fitness of best Student: 972.0


In [71]:
# Benchmark 4 - linear minimization with 8 variables and 3 non linear inequality and 3 linear inequality constraints
def objective_4(x):
    return x[0] + x[1] + x[2]

# Inequality constraint functions
def constraint_4(x):
    constraints = np.zeros(len(x))
    constraints[0] = -1 + 0.0025 * (x[3] + x[5])
    constraints[1] = -1 + 0.0025 * (x[4] + x[6] - x[3])
    constraints[2] = -1 + 0.01 * (x[7] - x[4])
    constraints[3] = - x[0] * x[5] + 833.33252 * x[3] + 100 * x[0] - 83333.333
    constraints[4] = - x[1] * x[6] + 1250 * x[4] + x[1] * x[3] - 1250 * x[3]
    constraints[5] = - x[2] * x[7] + 1250000 + x[2] * x[4] - 2500 * x[4]
    return constraints

# Initialize
max_iter = 2000 # No. of generations
n_pop = 50 # Population size
n_dim = 8 # No. of design variables
pop = np.random.uniform(-1000, 10000, size=(n_pop, n_dim)) # Range

t4 = TLBO(pop, f_obj = objective_4, f_cons = constraint_4)

print("No. of generation: ", max_iter)
print("Population Size: ", n_pop)
print("No. of design variables: ", n_dim)
print("\nBest Student found:")
print(t4)
fitness_value = objective_4(t4)
print("\nFitness of best Student:", fitness_value)

No. of generation:  2000
Population Size:  50
No. of design variables:  8

Best Student found:
[-1.         -1.         -1.          0.11013485 -1.         -1.
 -0.44185738  0.80476343]

Fitness of best Student: -3.0


In [72]:
# Benchmark 5 - quadratic maximization with 3 variables and 9**3 = 729 non linear inequality constraints
def objective_5(x):
    return (100 - (x[0] - 5)**2 - (x[1] - 5)**2 - (x[2] - 5)**2)/100

# Inequality constraint functions
def constraint_5(x):
    constraints = np.zeros(len(x))
    for i in range(1, 10):
      constraints[0] = (x[0] - i)**2 + (x[1] - i)**2 + (x[2] - i)**2
      return constraints

# Initialize
max_iter = 1000 # No. of generations
n_pop = 50 # Population size
n_dim = 3 # No. of design variables
pop = np.random.uniform(0, 10, size=(n_pop, n_dim)) # Range

t5 = TLBO(pop, f_obj = objective_5, f_cons = constraint_5)

print("No. of generation: ", max_iter)
print("Population Size: ", n_pop)
print("No. of design variables: ", n_dim)
print("\nBest Student found:")
print(t5)
fitness_value = objective_5(t5)
print("\nFitness of best Student:", fitness_value)

No. of generation:  1000
Population Size:  50
No. of design variables:  3

Best Student found:
[-1. -1. -1.]

Fitness of best Student: -0.08
